In [66]:
%matplotlib inline
import pandas
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np


pandas.set_option('display.max_columns', None)  
pandas.set_option('display.expand_frame_repr', False)
#pandas.set_option('max_colwidth', -1)
pandas.set_option('display.precision', 3)

In [67]:
df = pandas.read_csv('data.csv', sep=',', na_values="")
print(df[0:10])

       Date      Time                                         Tweet_Text  Type Media_Type                    Hashtags   Tweet_Id                                          Tweet_Url  twt_favourites_IS_THIS_LIKE_QUESTION_MARK  Retweets  Unnamed: 10  Unnamed: 11
0  16-11-11  15:26:37  Today we express our deepest gratitude to all ...  text      photo                   ThankAVet  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     127213     41112          NaN          NaN
1  16-11-11  13:33:35  Busy day planned in New York. Will soon be mak...  text        NaN                         NaN  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     141527     28654          NaN          NaN
2  16-11-11  11:14:20  Love the fact that the small groups of protest...  text        NaN                         NaN  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     183729 

In [68]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# Preparación de las herramientas de preprocesamiento de texto
nltk.download('stopwords')
stop = set(stopwords.words('english')) 
sno = SnowballStemmer('english') 

def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

# Procesamiento del texto de los tweets
final_string = []
for sent in df['Tweet_Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent) 
    for w in sent.split():
        cleaned_words = cleanpunc(w)
        if cleaned_words.isalpha() and len(cleaned_words) > 2:
            if cleaned_words.lower() not in stop:
                stemmed_word = sno.stem(cleaned_words.lower())
                filtered_sentence.append(stemmed_word)
    final_string.append(" ".join(filtered_sentence))

# Añadiendo la columna de tweets limpios al DataFrame
df['cleaned_tweet'] = final_string

# Visualización de los primeros registros del DataFrame modificado
print(df["cleaned_tweet"].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aleja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    today express deepest gratitud serv arm thankavet
1    busi day plan new soon make import decis peopl...
2    love fact small group protest last night passi...
3      open success presidenti profession incit unfair
4    fantast day met presid obama first realli good...
Name: cleaned_tweet, dtype: object


In [69]:

media_likes= df["twt_favourites_IS_THIS_LIKE_QUESTION_MARK"].median()
media_retweets=df["Retweets"].median()
df['Viral'] = df.apply(lambda x: "mucho" if (x['twt_favourites_IS_THIS_LIKE_QUESTION_MARK'] + x['Retweets']) >= (media_likes + media_retweets) * 3
                       else ("moderado" if (x['twt_favourites_IS_THIS_LIKE_QUESTION_MARK'] + x['Retweets']) >= (media_likes + media_retweets)*0.5
                             else "poco"), axis=1)

In [55]:
print(df["Viral"].value_counts())

Viral
moderado    3464
poco        2272
mucho       1639
Name: count, dtype: int64


In [ ]:
df.drop('Tweet_Text', axis=1, inplace=True)
df.drop('Tweet_Id', axis=1, inplace=True)
df.drop('Tweet_Url', axis=1, inplace=True)
df.drop('twt_favourites_IS_THIS_LIKE_QUESTION_MARK', axis=1, inplace=True)
df.drop('Retweets', axis=1, inplace=True)
df.drop('Unnamed: 10', axis=1, inplace=True)
df.drop('Unnamed: 11', axis=1, inplace=True)
y=df["Viral"].values
# Definir X como todas las columnas excepto 'Viral'
X= df.drop('Viral', axis=1).values


In [57]:
print(df[0:10])

       Date      Time  Type Media_Type                    Hashtags                                      cleaned_tweet  Viral
0  16-11-11  15:26:37  text      photo                   ThankAVet  today express deepest gratitud serv arm thankavet  mucho
1  16-11-11  13:33:35  text        NaN                         NaN  busi day plan new soon make import decis peopl...  mucho
2  16-11-11  11:14:20  text        NaN                         NaN  love fact small group protest last night passi...  mucho
3  16-11-11   2:19:44  text        NaN                         NaN    open success presidenti profession incit unfair  mucho
4  16-11-11   2:10:46  text        NaN                         NaN  fantast day met presid obama first realli good...  mucho
5  16-11-10  19:31:27  text      photo                         NaN             happi birthday marin corp thank servic  mucho
6  16-11-09  11:36:58  text        NaN                         NaN  beauti import even forgotten man woman never f...  mucho


## 3- Simple cross-validation

In [71]:
import sklearn.model_selection as cv 
(X_train, X_test,  y_train, y_test) = cv.train_test_split(X, y, test_size=.3, random_state=1)
print('Size training: ',X_train.shape)
print('Size test: ',X_test.shape)
print(X_test[:5])

Size training:  (5162, 6)
Size test:  (2213, 6)
[['16-04-25' '18:11:08' 'link' nan nan 'dead clinton trump']
 ['15-11-13' '20:18:12' 'text' nan nan
  'said four week ago putinnev said green separ piecesgreat rate']
 ['16-02-17' '3:04:22' 'text' nan nan
  'explain know beat hillari win state']
 ['16-01-15' '15:30:38' 'text' nan nan
  'lindsey graham embarrass fail run presid embarrass endors']
 ['16-02-16' '11:22:02' 'text' nan nan
  'new ppp poll trump rubio bush debat even stack rnc wonder']]


In [59]:
print( X_train[:, -1])

['great honor result cnn poll believ final result even better'
 'drug pour ice endors debat bigleaguetruth'
 'michigan gop poll trump rubio cruz kasich carson wow' ...
 'peopl tweet million low poll big hit ad'
 'thank great crowd support get vote let maga'
 'saw share want love hero cute']


In [72]:
## Conversion from text to Bag of Words representation as a table

from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
Bow_train = count_vectorizer.fit_transform( X_train[:, -1])
Bow_test = count_vectorizer.transform( X_test[:, -1])

feature_names = count_vectorizer.get_feature_names_out()
vocabulary = count_vectorizer.vocabulary_
print("Vocabulary:", vocabulary)

print('Shape of table for training: ',Bow_train.shape)
print('Shape of table for test: ',Bow_test.shape)
print( Bow_train[:10])

Vocabulary: {'great': 1567, 'honor': 1710, 'result': 2986, 'cnn': 674, 'poll': 2719, 'believ': 317, 'final': 1356, 'even': 1232, 'better': 345, 'drug': 1099, 'pour': 2739, 'ice': 1754, 'endors': 1181, 'debat': 899, 'bigleaguetruth': 354, 'michigan': 2271, 'gop': 1532, 'trump': 3727, 'rubio': 3065, 'cruz': 854, 'kasich': 1962, 'carson': 547, 'wow': 4058, 'kick': 1979, 'back': 252, 'boat': 394, 'bought': 423, 'lobbyist': 2108, 'money': 2329, 'claim': 642, 'book': 407, 'sale': 3096, 'profit': 2796, 'lol': 2113, 'interest': 1849, 'read': 2887, 'peggi': 2622, 'find': 1359, 'offens': 2495, 'goofi': 1529, 'elizabeth': 1161, 'sometim': 3342, 'refer': 2915, 'pretend': 2767, 'nativ': 2382, 'american': 120, 'get': 1494, 'fail': 1289, 'refus': 2923, 'use': 3854, 'name': 2371, 'given': 1506, 'proud': 2818, 'help': 1661, 'dishonest': 1023, 'start': 3415, 'busi': 490, 'littl': 2104, 'built': 481, 'best': 340, 'real': 2893, 'estat': 1224, 'asset': 211, 'amaz': 113, 'peopl': 2634, 'leav': 2053, 'left':

In [61]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Inicializar el CountVectorizer
count_vectorizer = CountVectorizer()

# Ajustar y transformar los datos
Bow_train = count_vectorizer.fit_transform(X_train[:, -1])
Bow_test = count_vectorizer.transform(X_test[:, -1])

# Obtener el vocabulario
vocabulary = count_vectorizer.vocabulary_

# Contar las frecuencias de las palabras en Bow_train
word_counts = np.asarray(Bow_train.sum(axis=0)).flatten()

# Obtener los índices de las 10 palabras más frecuentes
top_indices = word_counts.argsort()[-10:][::-1]

# Obtener las 10 palabras más frecuentes
top_words = [list(vocabulary.keys())[list(vocabulary.values()).index(i)] for i in top_indices]

# Mostrar las 10 palabras más frecuentes
print("Top 10 most frequent words:", top_words)

Top 10 most frequent words: ['trump', 'great', 'thank', 'hillari', 'makeamericagreatagain', 'america', 'make', 'get', 'new', 'poll']


In [ ]:
df_bow_train = pandas.DataFrame(Bow_train.toarray(), columns=feature_names)
print(df_bow_train)

      abandon  abc  abcwashington  abedin  aberdeen  abil  abl  aboard  abolish  abort  abraham  absente  absolut  abt  abu  abus  accept  access  accessori  accid  accolad  accompani  accomplish  accord  account  accus  achiev  acknowledg  acquir  across  act  actingmeek  action  activ  actual  ad  adam  add  addit  address  admin  administr  admir  admit  admonish  ador  adult  advanc  advantag  advertis  advic  advis  advisori  advoc  aerial  aesop  affair  affect  afford  afraid  african  afternoon  age  agent  aggress  agnstt  ago  agre  agreement  ahead  aid  ail  ailsa  aim  air  airbus  airlin  airplan  airport  airtim  airwav  aka  akbar  alabama  alaska  albert  album  alec  alejandro  alex  ali  alicia  alien  alisa  alisar  alisyn  aliv  alivebig  allah  alley  alli  allow  allyn  almost  alon  along  alongsid  alpha  alreadi  also  alway  alwaystrump  amateur  amaz  ambien  ambush  amen  amend  america  americafirst  american  americanprid  americansamoa  americash  amnest

# NAIVE BAYES

In [73]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(X[:, -1])
print(X.shape)
print(y.shape)

(7375, 4824)
(7375,)


In [75]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

# Convertir la matriz dispersa X a una matriz densa
X_dense = X.toarray()

# Configuración de la validación cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

# Inicializar el clasificador Naive Bayes
gnb = GaussianNB()

# Realizar validación cruzada
cv_scores = cross_val_score(gnb, X=X_dense, y=y, cv=cv)

# Mostrar la puntuación media
print(np.mean(cv_scores))

0.3464433192500174


### AQUI SE ESTA CALCULANDO LA MATRIZ DE CONFUSIÓN Y LA ACCURACY

In [78]:
from sklearn.model_selection import cross_val_predict  
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

predicted = cross_val_predict(GaussianNB(), X=X_dense, y=y,  cv=cv)  

print(confusion_matrix(y, predicted))
print(accuracy_score(y, predicted))

[[ 641 2202  621]
 [ 254 1232  153]
 [ 264 1326  682]]
0.3464406779661017


### AQUI SE ESTA CALCULANDO LA PRECISIÓN , RECALL, F1-SCORE

In [79]:
from sklearn.metrics import classification_report

print(classification_report(y, predicted))

              precision    recall  f1-score   support

    moderado       0.55      0.19      0.28      3464
       mucho       0.26      0.75      0.39      1639
        poco       0.47      0.30      0.37      2272

    accuracy                           0.35      7375
   macro avg       0.43      0.41      0.34      7375
weighted avg       0.46      0.35      0.33      7375



# KNN

In [ ]:
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds            # Per carregar mÃ©s facilment el dataset digits
import sklearn.model_selection as cv    # Pel Cross-validation
import sklearn.neighbors as nb 
knc = nb.KNeighborsClassifier()

# Train the classifier
knc.fit(Bow_train, y_train)

# Obtain accuracy score of learned classifier on test data
print(knc.score(Bow_test, y_test))


0.47401717126073206


In [ ]:
# More4 information with confussion matrix
from sklearn.metrics import confusion_matrix

y_pred = knc.predict(Bow_test)
print(sklearn.metrics.confusion_matrix(y_test, y_pred))

[[630  46 380]
 [273  64 147]
 [302  16 355]]


In [ ]:
# Obtain Recall, Precision and F-Measure for each class
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    moderado       0.52      0.60      0.56      1056
       mucho       0.51      0.13      0.21       484
        poco       0.40      0.53      0.46       673

    accuracy                           0.47      2213
   macro avg       0.48      0.42      0.41      2213
weighted avg       0.48      0.47      0.45      2213

